In [2]:
# import libraries
from bs4 import BeautifulSoup
import requests as req
import pickle
import re
import pandas as pd
import numpy as np
from io import BytesIO
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print(bs4.__version__)
print(req.__version__)
print(pd.__version__)
print(np.__version__)

In [ ]:
# find a spefic href tag to identify the car id-dl 
def car_model_href(href):
    return href and re.compile(r"(^(info.php).*)").search(href)

# creating lists for records
car_id_dl_map = {}
id_list = []
dl_list = []

# loop through all pages in sgcarmart used car listing
BRSR = 0
RPG = 100
for BRSR in range(0, 2000, 100): #100000
      url = f"https://www.sgcarmart.com/used_cars/listing.php?BRSR={BRSR}&RPG={RPG}"
      html_text = req.get(url)
      soup = BeautifulSoup(html_text.content, "lxml")
      listings = soup.body.find('div', {'class': 'listing_searchbar_position'}).p.string.replace(" ", "")
      cleaned_listings = re.sub("\D", '', listings)
      listings_count = int(cleaned_listings)
      # car models found per page
      car_model_list = soup.body.find('div', {'id':'content'}).find('form', {'name':'searchform'}).next_sibling.next_sibling.find_all(href=car_model_href, string=True)
      for model in car_model_list:
          id = str(model).partition('ID')[2].partition('&')[0].partition('=')[-1]
          dl = str(model).partition('DL')[2].partition('"')[0].partition('=')[-1]
          if id not in car_id_dl_map:
              car_id_dl_map[id] = dl
              id_list.append(id)
              dl_list.append(dl)

In [93]:
# Functions defined to scrape each features
def get_model(soup):
    model = soup.body.find('div', {'id':'contentblank'}).select('div:nth-of-type(2)')[0].select('div:nth-of-type(1)')[0].text.strip()
    return model

def get_price(soup):
    pr = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).find('tr').find_all('strong')[1]
    pr = str(pr).partition('/')[0].partition('>')[-1].partition('<')[0].strip()
    return pr

def get_depreciation(soup):
    dep = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(2)')[0].select('td:nth-of-type(2)')[0].find_all(string=re.compile("$"))
    dep = dep[0].replace(" ", "").strip().partition('/')[0]
    return dep

def get_registrationDate(soup):
    regDate = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(2)')[0].select('td:nth-of-type(4)')[0].find_all(string=True)
    reg = regDate[0].strip()
    return reg

def get_mileage(soup):
    mi = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('div:nth-of-type(1)')[0].find('div', {'class':'row_info'}).string
    mi = mi.strip()
    return mi

def get_roadTax(soup):
    roadTax = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].find('td').select('div:nth-of-type(1)')[0].next_sibling.next_sibling.find('div', {'class':'row_info'}).string
    roadTax = roadTax.strip()
    return roadTax

def get_deregistrationValue(soup):
    dereg = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].find('td').select('div:nth-of-type(3)')[0].find('div', {'class':'row_info'}).find(string=re.compile("$"))
    dereg = dereg.partition("as")[0].strip()
    return dereg

def get_coe(soup):
    coe = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].find('td').select('div:nth-of-type(4)')[0].find('div', {'class':'row_info'}).string
    coe = coe.strip()
    return coe

def get_engineCap(soup):
    eng = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].find('td').select('div:nth-of-type(5)')[0].find('div', {'class':'row_info'}).string
    eng = eng.strip()
    return eng

def get_curbWeight(soup):
    weight = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].find('td').select('div:nth-of-type(6)')[0].find('div', {'class':'row_info'}).string
    weight = weight.strip()
    return weight

def get_manufacturedYear(soup):
    year = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(1)')[0].find('div', {'class':'row_info'}).string
    year = year.strip()
    return year

def get_transmission(soup):
    trans = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].find('div', {'class':'eachInfo'}).next_sibling.next_sibling.find('div', {'class':'row_info'}).string
    trans = trans.strip()
    return trans

def get_omv(soup):
    tag = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(3)')[0].find('div', {'class':'row_title'}).text.strip()
    if tag == 'OMV':
        omv = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(3)')[0].find('div', {'class':'row_info'}).string
        omv = omv.strip()
    elif tag == 'Fuel Type':
        omv = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(4)')[0].find('div', {'class':'row_info'}).string
        omv = omv.strip()
    else:
        omv = 'NA'
    return omv

def get_arf(soup):
    tag = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(4)')[0].find('div', {'class':'row_title'}).text.strip()
    if tag == 'ARF':
        arf = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(4)')[0].find('div', {'class':'row_info'}).string
        arf = arf.strip()
    elif tag == 'OMV':
        arf = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(5)')[0].find('div', {'class':'row_info'}).string
        arf = arf.strip()
    else:
        arf = 'NA'
    return arf

def get_power(soup):
    tag = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(5)')[0].find('div', {'class':'row_title'}).text.strip()
    if tag == 'Power':
        power = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(5)')[0].find('div', {'class':'row_info'}).string
        power = power.strip()
    elif tag == 'ARF':
        power = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(6)')[0].find('div', {'class':'row_info'}).string
        power = power.strip()
    else:
        power = 'NA'
    return power

def get_number_of_owner(soup):
    tag = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(6)')[0].find('div', {'class':'row_title'}).text.strip()
    if tag == 'No. of Owners':
        owner = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(6)')[0].find('div', {'class':'row_info'}).string
        owner = owner.strip()
    elif tag == 'Power':
        owner = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(7)')[0].find('div', {'class':'row_info'}).string
        owner = owner.strip()
    else:
        owner = 'NA'
    return owner

def get_type(soup):
    try:
        typ = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(4)')[0].find('a').text
        typ = typ.strip()
    except:
        typ = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].find('td').select('div:nth-of-type(7)')[0].find('div', {'class':'row_info'}).text
        typ = typ.strip()   
    return typ

def get_all_other_info(soup):
    
    # list of other info should return in this sequence [features, accessories, descriptions, category, status]
    other_info = ['NA', 'NA', 'NA', 'NA', 'NA']
    
    # get the rows info
    for i in range(5):
        position = i + 5
        row = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(%d)'%position)[0].find('td').text
        cleaned_row = ' '.join([t for t in (row.replace('\r', '').replace('\n','').strip().split(" ")) if len(t) > 0])
        if (cleaned_row.split(" ")[0] == 'Features'):
            other_info[0] = cleaned_row
        elif (cleaned_row.split(" ")[0] == 'Accessories'):
            other_info[1] = cleaned_row
        elif (cleaned_row.split(" ")[0] == 'Description'):
            other_info[2] = cleaned_row
        elif (cleaned_row.split(" ")[0] == 'Category'):
            other_info[3] = cleaned_row
        elif (cleaned_row.split(" ")[0] == 'Status'):
            other_info[4] = cleaned_row
            
    # somehow category and status data might not be read in do continue with the following
    if (other_info[1] == 'NA'):
        cat = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(7)')[0].find('td').text
        cleaned_cat = ' '.join([t for t in (cat.replace('\r', '').replace('\n','').replace('Status', '').strip().split(" ")) if len(t) > 0])
        other_info[3] = cleaned_cat
        
        stat = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(8)')[0].find('td').text
        cleaned_stat = ' '.join([t for t in (stat.replace('\r', '').replace('\n','').replace('Status', '').strip().split(" ")) if len(t) > 0])
        other_info[4] = cleaned_stat
    else:
        cat = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(8)')[0].find('td').text
        cleaned_cat = ' '.join([t for t in (cat.replace('\r', '').replace('\n','').replace('Status', '').strip().split(" ")) if len(t) > 0])
        other_info[3] = cleaned_cat
        
        stat = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(9)')[0].find('td').text
        cleaned_stat = ' '.join([t for t in (stat.replace('\r', '').replace('\n','').replace('Status', '').strip().split(" ")) if len(t) > 0])
        other_info[4] = cleaned_stat
        
    return other_info

def get_image(soup):
    try:
        image = soup.body.find('div', {'id':'contentblank'}).select('div:nth-of-type(2)')[0].select('div:nth-of-type(5)')[0].select('div:nth-of-type(1)')[0].next_sibling.next_sibling.findAll('img')[0].attrs['src']
    except:
        image = None
        pass
    return image

def get_postUpdate_date(soup):
    posted = ''
    updated = ''
    try:
        date = soup.body.find('div', {'id':'usedcar_postdate'}).get_text(strip=True).replace('\xa0', '').replace('\r', '').replace('\n', ''). replace('\t', '')
        posted = re.search('(?<=Posted on:).*?(?=\|)', date)[0].strip()
        updated = re.search('(?<=Updated on:).*', date)[0].strip()
    except:
        pass
    
    return posted, updated

In [94]:
# scrape html data from text file
# read a text file in format of .txt which contains html text
def feature_scraping(filePath):

    # read the file
    with open(filePath, 'r') as file:
        html_content = file.read()
        
    print(filePath)

    # read the html content with BS4
    soup = BeautifulSoup(html.content, 'lxml')

    # scrape standard features
    model = get_model(soup)
    price = get_price(soup)
    depreciation = get_depreciation(soup)
    reg_date = get_registrationDate(soup)
    manufactured_year = get_manufacturedYear(soup)
    mileage = get_mileage(soup)
    road_tax = get_roadTax(soup)
    transmission = get_transmission(soup)
    dereg_value = get_deregistrationValue(soup)
    omv = get_omv(soup)
    coe = get_coe(soup)
    arf = get_arf(soup)
    engine_cap = get_engineCap(soup)
    power = get_power(soup)
    curb_weight = get_curbWeight(soup)
    number_of_owner = get_number_of_owner(soup)
    types = get_type(soup)
    images = get_image(soup)

    # scrape other features
    other_info  = get_all_other_info(soup)
    features = other_info[0]
    accessories = other_info[1]
    descriptions = other_info[2]
    category = other_info[3]
    status = other_info[4]
    
    # scrape posted and updated date
    posted_date = get_postUpdate_date(soup)[0]
    updated_date = get_postUpdate_date(soup)[1]

    # download the image to folder via the image link
    # use pillow to read the btye string
    response = req.get(images)
    if response.status_code == 200:
        img = Image.open(BytesIO(response.content))
        img.save(f'/Users/eesoonhang/Library/CloudStorage/OneDrive-Personal/04. MTech_Y2_Sem1/4.0 Project/Resources/{model}.jpg') # specific your full file diretory, including filename here

    return model, price, depreciation, reg_date, manufactured_year, mileage, road_tax, transmission, dereg_value, omv, coe, arf, engine_cap, power, curb_weight, number_of_owner, types, features, accessories, descriptions, category, status, images, posted_date, updated_date

In [95]:
# combine the feature scraping function with the pathList
# NOTE: do replace [path] in the map function with a list of file directory\
# url = 'https://www.sgcarmart.com/used_cars/info.php?ID=1182894&DL=1000'
path = '/Users/eesoonhang/Downloads/test1.txt'
%time sgCarMart_features = list(map(feature_scraping, [path]))

CPU times: user 168 ms, sys: 4.39 ms, total: 172 ms
Wall time: 877 ms


In [96]:
# transform to dataframe
df = pd.DataFrame(sgCarMart_features,
                 columns = ['model', 'price', 'depreciation', 'registration_date', 'manufactured_year', 'mileage', 'road_tax', 'transmission', 'deregistration_value', 'omv', 'coe', 'arf', 'engine_cap', 'power', 'curb_weight', 'number_of_owner', 'types', 'features', 'accessories', 'descriptions', 'category', 'status', 'images', 'posted_date', 'updated_date'])
df.head()

,model,price,depreciation,registration_date,manufactured_year,mileage,road_tax,transmission,deregistration_value,omv,...,number_of_owner,types,features,accessories,descriptions,category,status,images,posted_date,updated_date
0,Land Rover Range Rover Sport Diesel 3.0A,"$769,998",N.A.,21-Jun-2013,2012,"125,595 km (12.8k /yr)","$3,376 /yr",Auto,"$60,243","$80,849",...,4,SUV,"Features Exporter Or Buyer, Feel Free To Welco...",NA,Description Full Tank Diesel Can Drive Up Till...,"CategoryPARF Car, Direct Owner Sale",Available for sale. Shortlist this car to get ...,https://i.i-sgcm.com/cars_used/202303/1182894_...,26-Mar-2023,13-Apr-2023


In [ ]:
client = storage.Client()
export_bucket = client.get_bucket("beadstore")

In [ ]:
# write scrape data as csv into Google Cloud Storage
df.to_csv()
export_bucket.blob("sgcarmart_usedcar_info {0}.csv".format(datetime.datetime.now()strftime('%Y-%m-%d %H_%M_%S'))).upload_from_string(df_all.to_csv(), 'text/csv')
# df_all.to_csv('gs://beadstore/sgcarmart_usedcar_info/sgcarmart_usedcar_info.csv', index = True)